## SAKILA DATABASE: Data Analysis and Visualisation 

*Notebook by Vaishnavi Sharma*

![](http://https://www.google.com/images/branding/googlelogo/1x/googlelogo_light_color_272x92dp.png)

**About Sakila Database:** The Sakila sample database was initially developed by Mike Hillyer, a former member of the MySQL AB documentation team. It is intended to provide a standard schema that can be used for examples in books, tutorials, articles, samples, and so forth. The Sakila database is a nicely normalised schema modelling a DVD rental store, featuring things like films, actors, film-actor relationships, and a central inventory table that connects films, stores, and rentals.

**About the notebook:**The notebook connects to the SAKILA database and explores the relation between various attributes of different tables in the database with the help of SQL queries and Data visualization.

* Tools used: SQL, pandas, matplotlib, numpy, seaborn
* Number of SQL queries: 18
* Visualizations used: pie chart, donut chart, bar graph, line chart, boxplot.
* Some of the questions answered:
                                - Number of films of each rating.
                                - Total number of unavailable films.
                                - Total monthly sales.
                                - Sales of different stores.
                                - Replacement cost and stock size.
                                
                                    

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt

In [ ]:
pd.options.display.max_rows = 20 # maximum number of rows to display

In [ ]:
# connecting to the salika database
database_path = "../input/sqlite-sakila-sample-database/sqlite-sakila.db"
connection = sqlite3.connect(database_path)

In [ ]:
# Displaying all the tables in the Sakila Database
tables = pd.read_sql("""

SELECT *
FROM sqlite_master
WHERE type = 'table';

""", connection)

tables

In [ ]:
# The film table is a list of all films potentially in stock in the stores. The actual in-stock copies 
# of each film are represented in the inventory table.

# Q. How many films of each rating are present in 'film' table?
film_ratings = pd.read_sql("""

SELECT 
    rating, COUNT(film_id) AS film_count
    FROM film
    GROUP BY rating
    ORDER BY COUNT(film_id) DESC

""", connection)

film_ratings 

In [ ]:
#pie chart for film_ratings 

fig, ax = plt.subplots(figsize = (8,8))

def func(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return "{:,.0f}\n({:.0f}%)".format(absolute, pct)


wedges, texts, autotexts = ax.pie(film_ratings["film_count"],
       labels = film_ratings["rating"],
       autopct=lambda pct: func(pct, film_ratings["film_count"]),
       textprops=dict(color="w", fontsize = 13, fontweight = "bold"))
ax.legend(wedges, film_ratings["rating"],
          title="Ratings",
          fontsize = 10,
          loc="center left",
          bbox_to_anchor=(1, 0.2, 0.5, 1))
ax.set_title("Film Ratings", fontsize = 15, color='#ffffff')
plt.rcParams['legend.title_fontsize'] = 12

plt.show()

In [ ]:
# The inventory table contains one row for each copy of a given film in a given store.

# Q. How many films(DISTINCT) of each rating are in stock?

film_rating_inventory = pd.read_sql("""

SELECT 
    film.rating, COUNT(DISTINCT inventory.film_id) AS film_count
    FROM film JOIN inventory
    ON film.film_id = inventory.film_id
    GROUP BY film.rating
    ORDER BY COUNT(inventory.film_id) DESC

""", connection)

film_rating_inventory

In [ ]:
# Donut graph for film rating count of films in inventory.

fig, ax = plt.subplots(figsize = (5,5))

def func(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return "{:,.0f} films".format(absolute, pct)

 
wedges, texts, autotexts = ax.pie(film_rating_inventory["film_count"],
       labels = film_ratings["rating"],
       autopct=lambda pct: func(pct, film_rating_inventory["film_count"]),
       explode = (0.05, 0.05, 0.05, 0.05, 0.05),
       textprops=dict(color="#00fff6", fontsize = 13, fontweight = "bold"))
ax.legend(wedges, film_rating_inventory["rating"],
          title="Ratings",
          fontsize = 10,
          loc="center left",
          bbox_to_anchor=(1, 0.2, 0.5, 1))
ax.set_title("Film Ratings", fontsize = 10, color='#ffffff')
plt.rcParams['legend.title_fontsize'] = 10

# Adding Circle in Pie chart
centre_circle = plt.Circle((0, 0), 0.70, fc='#ffffff')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.show()

In [ ]:
# Q. How many films are unvailable at the stores?
# A. Total 42 films in 'film' table are not in 'inventory' table.

pd.read_sql("""

SELECT
	COUNT(film_id) AS films_unavailable
	FROM film 
	WHERE film_id NOT IN (SELECT
							film_id 
							FROM inventory)	

""", connection)

In [ ]:
# Q. In which cities are the stores located?
# A. There are only 2 stores one in Lethbridge and one in Woodridge.

pd.read_sql("""

SELECT
	city.city,  COUNT(store.store_id) AS number_of_stores
	FROM store JOIN address JOIN city
	ON store.address_id = address.address_id
	AND address.city_id = city.city_id
	GROUP BY address.city_id

""", connection)

In [ ]:
# Q. What is the average rental duration of films of each category?
# A. The average rental duration for every catergory is approx 5 days.

avg_rental_duration_category = pd.read_sql("""
	
SELECT 
	category.category_id,category.name AS category_name, AVG(film.rental_duration) AS avg_rental_duration
	FROM film JOIN film_category JOIN category
	ON film.film_id = film_category.film_id 
	AND film_category.category_id = category.category_id
	GROUP BY category.category_id
	ORDER BY category_name
    
""", connection)

avg_rental_duration_category

In [ ]:
# bar chart of avg_rental_duration_category

fig, ax = plt.subplots(figsize = (16,4))

ypos = np.arange(len(avg_rental_duration_category["avg_rental_duration"]))
bars = ax.bar(ypos, avg_rental_duration_category["avg_rental_duration"], width = 0.50)
ax.set_xticks(ypos)
ax.set_xticklabels(avg_rental_duration_category["category_name"])
ax.set_ylim(ymax = 6)
ax.set_title("Average rental duration of films in different categories", fontsize = 14)
ax.set_ylabel("Number of days", fontsize = 12)


plt.show()

In [ ]:
# Q. Which 3 category films are most common?
# A. Sports, Foreign and Family categories have most number of films.

pd.read_sql("""
	
SELECT 
	category.name, COUNT(film_category.film_id) AS film_cnt
	FROM film_category JOIN category
	ON film_category.category_id = category.category_id
	GROUP BY category.name
	ORDER BY COUNT(film_category.film_id) DESC
	LIMIT 3
    
""", connection)

In [ ]:
# Q. How many films of different languages?
# A. there are total 1000 films, all in english language.

pd.read_sql("""
	
SELECT
	language.name, COUNT(film.film_id)
	FROM film JOIN language
	ON film.language_id = language.language_id
    GROUP BY language.name
	
""", connection)

In [ ]:
# Q. Which actors have performed films of all 16 categories?
# A. total 11 actors have worked in all 16 categories of films.

pd.read_sql("""
	
SELECT 
	sub_table.actorid, actor.first_name, actor.last_name, sub_table.categories_cnt
	FROM actor JOIN (SELECT 
						film_actor.actor_id AS actorid, COUNT(DISTINCT film_category.category_id) AS categories_cnt
						FROM film_actor JOIN film JOIN film_category
						ON film_actor.film_id = film.film_id
						AND film.film_id = film_category.film_id
						GROUP BY film_actor.actor_id
						HAVING COUNT(DISTINCT film_category.category_id) >= 16) AS sub_table
	ON actor.actor_id = sub_table.actorid
	
""", connection)

In [ ]:
# Q. Which actor, amongst the actors who have worked in all 16 film categories, has performed in most numer of films?
# A. GINA DEGENERES has performed in most number of films amongst the actors who have performed in all categories.

pd.read_sql("""
	
SELECT 
	actor.actor_id, actor.first_name, actor.last_name, COUNT(film_actor.film_id) AS films_count
	FROM actor JOIN film_actor
	ON actor.actor_id = film_actor.actor_id
	WHERE actor.actor_id IN (SELECT 
						film_actor.actor_id
						FROM film_actor JOIN film JOIN film_category
						ON film_actor.film_id = film.film_id
						AND film.film_id = film_category.film_id
						GROUP BY film_actor.actor_id
						HAVING COUNT(DISTINCT film_category.category_id) >= 16)
	GROUP BY actor.actor_id
    ORDER BY COUNT(film_actor.film_id) DESC
    LIMIT 1
	
""", connection)

In [ ]:
# What were the total sales per month?

total_m_sale = pd.read_sql("""
	
SELECT  
    strftime('%Y-%m', payment_date) AS Date,
    ROUND(SUM(amount), 0) AS Sales
    FROM payment
    GROUP BY Date
    ORDER BY Date ASC
	
""", connection)

total_m_sale

**!! As there is a gap of 6 months between Aug-2005 and Feb-2006, we will exclude 2006-02 sales month from the line graph !!**

In [ ]:
# line graph for total sales per month (EXCLUDING FEB-2006 SALES)

sales_per_month = total_m_sale.iloc[0:4].set_index("Date") # Excluding the 2006-02 sales

fig, ax = plt.subplots(figsize = (10,5))
ax.plot(sales_per_month,"g*-")
ax.set_ylim(ymin = 0, ymax = 30000)
ax.set_ylabel("Sales")
ax.set_xlabel("Date")

for date, sales in sales_per_month["Sales"].items():
    ax.annotate(str("${:,.0f}".format(sales)),
                xy=(date, sales))

plt.show()

In [ ]:
# Q. Make line graphs to compare monthly sales of each store.
# A. Since we now know that there is discontuinity of sales data from 2005-08 to 2006-02, we will exclude 2006-02 sales 
#    in the SQL query itself.

sale_per_store = pd.read_sql("""
	
SELECT 
	c.store_id AS store, strftime('%Y-%m', sub_table.payment_date) AS Date, ROUND(SUM(sub_table.amount),0) AS sales
	FROM customer c JOIN (SELECT   
							customer_id,
							payment_date,
							amount
							FROM payment
							) AS sub_table
	ON c.customer_id = sub_table.customer_id
    WHERE Date < '2006-01'
	GROUP BY c.store_id, Date
	ORDER BY c.store_id, Date
	
""", connection)

sale_per_store
# We will make line chart for the following table.

In [ ]:
plt.subplots(figsize=(8, 8))
sns.lineplot(data=sale_per_store, x="Date", y="sales", hue="store", palette="tab10", )
plt.show()

In [ ]:
# We see that sales of store 1 were consistently higher than sales of store 2.

In [ ]:
# Q. Make pivot table for monthly sales of the stores.

sales_store = pd.read_sql("""

SELECT 
	strftime('%Y-%m', sub_table.payment_date) AS Date, 
    c.store_id AS store,
    sub_table.amount AS sales_data
	FROM customer c JOIN (SELECT  
							customer_id,
							payment_date,
							amount
							FROM payment
							) AS sub_table
	ON c.customer_id = sub_table.customer_id
	
""", connection)


In [ ]:
pivottable = pd.pivot_table(sales_store, index=sales_store['Date'], columns=sales_store['store'], aggfunc=np.sum)
pivottable

In [ ]:
# Q. What are the 5 'G' rated films that have lowest replacement cost and atmost 3 copies of itself in the inventory?

pd.read_sql("""

SELECT 
	DISTINCT f.film_id, f.title, replacement_cost, COUNT(i.film_id) AS copies_in_inv
	FROM film f JOIN inventory i
	ON f.film_id = i.film_id
    WHERE rating = 'G'
    GROUP BY f.film_id, f.title
    HAVING copies_in_inv <= 3
	ORDER BY replacement_cost
	LIMIT 5
    
""", connection)

In [ ]:
# Q. What is the category id of 'Foreign' category?
# A. 9
pd.read_sql("""

SELECT c.name, fc.category_id from film_category fc JOIN category c
on fc.category_id = c.category_id
where c.name = 'Foreign'
LIMIT 1
""", connection)

In [ ]:
# Q. Compare the average replacement cost for films that are 'Foreign' category and flims that are not foreign category.
# A. Films of foreign category have lower mean replacement cost than mean replacement cost of films that are not foreign category.
replacement_cost_category = pd.read_sql("""

SELECT
	'Foriegn', AVG(f.replacement_cost) AS avg_replacement_cost
	FROM film f JOIN film_category fc
	ON f.film_id = fc.film_id
	WHERE fc.category_id = 9
UNION
SELECT
	'Non Foreign',AVG(f.replacement_cost) AS avg_replacement_cost
	FROM film f JOIN film_category fc
	ON f.film_id = fc.film_id
	WHERE fc.category_id != 9
 
""", connection)
replacement_cost_category

In [ ]:
# Q. Make Boxplot of replacement_cost of films

replacement_c = pd.read_sql("""    
SELECT 
    replacement_cost
    FROM film    
""", connection)

# making boxplot for replacement_c
replacement_c.boxplot(color='g', vert=False, grid=False, figsize=(12,4))

In [ ]:
# Q. Make boxplots for replacement cost of films of different ratings

replacement_c_rating = pd.read_sql("""  
SELECT 
    rating, replacement_cost
    FROM film    
""", connection)

# making boxplot for replacement_c_rating
replacement_c_rating[['replacement_cost' , 'rating']].boxplot(by='rating', figsize=(12,6), grid=False)

***THE END***